# Control Flow Protection by Cryptographic Instruction Chaining



A Python proof-of-concept implementation of the secure execution environment described in “Control Flow Protection by Cryptographic Instruction Chaining” 

---

## Overview

This repository contains a standalone Jupyter Notebook (`control_flow_protection.ipynb`) implementing:

- **Instruction encryption & chaining** using per-instruction key derivation  
- **Authenticated encryption** (AES-CFB + HMAC-SHA256) for instructions, registers, and memory  
- **Control-flow handling** (branches, jumps, calls/returns) with cryptographic binding  
- A **benchmark harness** measuring crypto-primitive throughput, memory-encrypt throughput, and per-instruction latency  

---

## Paper Summary

> **Control Flow Protection by Cryptographic Instruction Chaining**
> Shahzad Ahmad, Stefan Rass, Maksim Goman, Manfred Schlägl, Daniel Große.  
> LIT Secure and Correct Systems Lab, Johannes Kepler University Linz, Austria (April 30, 2025).
> We present a novel secure execution environment that provides comprehensive protection for program execution through a unified cryptographic approach. Our construction employs authenticated encryption (ensuring confidentiality, integrity, and correct ordering), and introduces a key-chaining mechanism that binds consecutive instructions to prevent reordering and replay attacks. Specialized handling for control-flow operations preserves security guarantees across complex execution paths. A Python proof-of-concept validates practicality, and performance analysis quantifies computational overhead.

Key contributions:

1. **Cryptographic binding of instructions** via fresh per-instruction keys  
2. **Authenticated encryption** for instructions, registers, and memory  
3. **Secure control-flow**: BEQ, JMP, CALL/RET handled with deterministic key derivation  
4. **Performance evaluation** demonstrating feasibility and overhead  

---

## Disclaimer

> **This code is provided “as-is” for research and educational purposes only.**  
> While it demonstrates the core cryptographic chaining concepts, it is **not** hardened for production use.  
> Do **not** run untrusted code under this environment without a full security audit. 

---

## Benchmark Results

Below is a summary of the end-to-end timings observed on a typical Python 3.x setup:

| **Operation**               | **Measured**         | **Derived Metric**                                            |
|-----------------------------|----------------------|---------------------------------------------------------------|
| Encrypt + MAC (1 KiB)       | 0.0321 s total       | Avg = 32.1 µs per 1 KiB<br>Throughput ≈ 31.9 MiB/s            |
| Decrypt + MAC (1 KiB)       | 0.0314 s total       | Avg = 31.4 µs per 1 KiB<br>Throughput ≈ 32.8 MiB/s            |
| Memory encrypt (32 B)       | 0.0002 s total       | Throughput ≈ 1.91 × 10⁵ B/s                                   |
| Instruction execution       | 0.0131 s for 18 instr| Avg = 728 µs per instr<br>Rate ≈ 1 374 instr/s                |

---

## Usage

1. Clone this repo  
2. Open `control_flow_protection.ipynb` in Jupyter or Colab  
3. Run all cells to see the implementation and benchmark output  

---

## License

This work has partially been supported by the LIT Secure and Correct Systems Lab funded by the State of Upper Austria.

In [5]:
import os
import hashlib
import hmac
import time
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

# Constants
KEY_SIZE     = 32       # bytes
BLOCK_SIZE   = 16       # bytes (AES block)
MAC_SIZE     = 32       # bytes (SHA-256 HMAC)
MEMORY_SIZE = 1024      # Size of memory in bytes

# Utility functions
def generate_key():
    return os.urandom(KEY_SIZE)

def encrypt(data: bytes, key: bytes) -> bytes:
    iv = os.urandom(BLOCK_SIZE)
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    return iv + encryptor.update(data) + encryptor.finalize()

def decrypt(ciphertext: bytes, key: bytes) -> bytes:
    iv, ct = ciphertext[:BLOCK_SIZE], ciphertext[BLOCK_SIZE:]
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    return decryptor.update(ct) + decryptor.finalize()

def compute_mac(data: bytes, key: bytes) -> bytes:
    return hmac.new(key, data, hashlib.sha256).digest()

def encrypt_with_mac(data: bytes, key: bytes) -> bytes:
    mac = compute_mac(data, key)
    return encrypt(data + mac, key)

def decrypt_with_mac(ct: bytes, key: bytes) -> bytes:
    pt_mac = decrypt(ct, key)
    data, mac_received = pt_mac[:-MAC_SIZE], pt_mac[-MAC_SIZE:]
    mac_expected = compute_mac(data, key)
    if not hmac.compare_digest(mac_received, mac_expected):
        raise ValueError("MAC verification failed")
    return data

def encrypt_memory(memory, key):
    encrypted_memory = {}
    for addr, value in memory.items():
        vb = value.to_bytes(8, byteorder='big', signed=True)
        encrypted_memory[addr] = encrypt_with_mac(vb, key)
    return encrypted_memory

def decrypt_memory(emem, key):
    dm = {}
    for addr, vb in emem.items():
        val = int.from_bytes(decrypt_with_mac(vb, key), byteorder='big', signed=True)
        dm[addr] = val
    return dm

# Benchmarking harness
def benchmark_op(func, data: bytes, key: bytes, iters: int = 1000):
    """Return (total_time_s, avg_time_us)."""
    t0 = time.perf_counter()
    for _ in range(iters):
        func(data, key)
    t1 = time.perf_counter()
    total = t1 - t0
    avg_us = (total / iters) * 1e6
    return total, avg_us

def benchmark_primitives():
    key = generate_key()
    sample = os.urandom(1024)  # 1 KiB

    enc_tot, enc_avg = benchmark_op(encrypt_with_mac, sample, key)
    print(f"[Encrypt+MAC]   Total: {enc_tot:.4f}s | Avg: {enc_avg:.1f} µs per 1 KiB")

    ct = encrypt_with_mac(sample, key)
    dec_tot, dec_avg = benchmark_op(decrypt_with_mac, ct, key)
    print(f"[Decrypt+MAC]   Total: {dec_tot:.4f}s | Avg: {dec_avg:.1f} µs per 1 KiB")


# Instruction functions
def mov(registers, dest, value):
    if isinstance(value, str):
        value = registers[value]
    registers[dest] = value

def add(registers, dest, src):
    if isinstance(src, str):
        src = registers[src]
    registers[dest] += src

def subtract(registers, dest, src):
    if isinstance(src, str):
        src = registers[src]
    registers[dest] -= src

def multiply(registers, dest, src):
    if isinstance(src, str):
        src = registers[src]
    registers[dest] *= src

def divide(registers, dest, src):
    if isinstance(src, str):
        src = registers[src]
    if src == 0:
        raise ValueError("Division by zero")
    registers[dest] //= src

def xor(registers, dest, src):
    if isinstance(src, str):
        src = registers[src]
    registers[dest] ^= src

def logical_and(registers, dest, src):
    if isinstance(src, str):
        src = registers[src]
    registers[dest] &= src

def logical_or(registers, dest, src):
    if isinstance(src, str):
        src = registers[src]
    registers[dest] |= src

def logical_not(registers, dest):
    registers[dest] = ~registers[dest]

def shl(registers, dest, value):
    if isinstance(value, str):
        value = registers[value]
    registers[dest] <<= value

def shr(registers, dest, value):
    if isinstance(value, str):
        value = registers[value]
    registers[dest] >>= value

def cmp(registers, reg, value):
    if isinstance(value, str):
        value = registers[value]
    return registers[reg] == value

def inc(registers, dest):
    registers[dest] += 1

def dec(registers, dest):
    registers[dest] -= 1

def nop(registers):
    # No operation
    pass

def load(registers, dest, address, memory):
    if isinstance(address, str):
        address = registers[address]
    registers[dest] = memory.get(address, 0)  # Default to 0 if address not found

def store(registers, src, address, memory):
    if isinstance(address, str):
        address = registers[address]
    if isinstance(src, str):
        src = registers[src]
    memory[address] = src

def encrypt_memory(memory, key):
    encrypted_memory = {}
    for addr, value in memory.items():
        value_bytes = value.to_bytes(8, byteorder='big', signed=True)
        encrypted_memory[addr] = encrypt_with_mac(value_bytes, key)
    return encrypted_memory

def decrypt_memory(encrypted_memory, key):
    decrypted_memory = {}
    for addr, value in encrypted_memory.items():
        decrypted_value = int.from_bytes(decrypt_with_mac(value, key), byteorder='big', signed=True)
        decrypted_memory[addr] = decrypted_value
    return decrypted_memory

def encrypt_registers(registers, key):
    encrypted_registers = {}
    for reg, value in registers.items():
        if isinstance(value, int):
            value_bytes = value.to_bytes(8, byteorder='big', signed=True)
            encrypted_registers[reg] = encrypt_with_mac(value_bytes, key)
        else:
            encrypted_registers[reg] = value
    return encrypted_registers

def decrypt_registers(registers, key):
    decrypted_registers = {}
    for reg, value in registers.items():
        if isinstance(value, bytes):
            decrypted_value = int.from_bytes(decrypt_with_mac(value, key), byteorder='big', signed=True)
            decrypted_registers[reg] = decrypted_value
        else:
            decrypted_registers[reg] = value
    return decrypted_registers

def encrypt_instruction_pointer(ip, key):
    ip_bytes = ip.to_bytes(8, byteorder='big', signed=False)
    iv = os.urandom(BLOCK_SIZE)
    return iv + encrypt_with_mac(ip_bytes, key)

def decrypt_instruction_pointer(encrypted_ip, key):
    if len(encrypted_ip) < BLOCK_SIZE * 2:  # At least IV + one block of ciphertext
        raise ValueError(f"Encrypted IP is too short: {len(encrypted_ip)} bytes")
    iv = encrypted_ip[:BLOCK_SIZE]
    ciphertext = encrypted_ip[BLOCK_SIZE:]
    decrypted_ip_bytes = decrypt_with_mac(ciphertext, key)
    return int.from_bytes(decrypted_ip_bytes, byteorder='big', signed=False)
    
def encrypt_labels(labels, master_key):
    encrypted_labels = {}
    for label, index in labels.items():
        label_bytes = label.encode()
        index_bytes = index.to_bytes(8, byteorder='big')
        encrypted_data = encrypt_with_mac(label_bytes + index_bytes, master_key)
        encrypted_labels[label] = encrypted_data
    return encrypted_labels

def decrypt_label(encrypted_label, master_key):
    decrypted_data = decrypt_with_mac(encrypted_label, master_key)
    label = decrypted_data[:-8].decode()
    index = int.from_bytes(decrypted_data[-8:], byteorder='big')
    return label, index

def log_instruction(instruction, registers_before, registers_after, kprev, knext):
    print("\n" + "="*50)
    print(f"Instruction: {instruction}")
    print(f"Kprev: {kprev.hex() if kprev else 'None'}")
    print(f"Knext: {knext.hex() if isinstance(knext, bytes) else knext}")
    print("Registers Before (decrypted):")
    for reg, value in registers_before.items():
        print(f"  {reg}: {value}")
    print("Registers After (encrypted):")
    for reg, value in registers_after.items():
        print(f"  {reg}: {value.hex() if isinstance(value, bytes) else value}")
    print("="*50)


def prepare_instructions(instructions, master_key):
    labels = {}
    encrypted_ips = {}
    
    print("\nInitial key assignment:")
    # First pass: Assign labels and generate keys
    for i, instr in enumerate(instructions):
        if len(instr["instr"]) == 1 and instr["instr"][0].endswith(':'):
            labels[instr["instr"][0][:-1]] = i
        instr["Knext"] = generate_key()
        if i > 0:
            instr["Kprev"] = instructions[i-1]["Knext"]
        else:
            instr["Kprev"] = master_key
        print(f"Instruction {i}: {instr['instr']}")
        print(f"  Kprev: {instr['Kprev'].hex()}")
        print(f"  Knext: {instr['Knext'].hex()}")

        # Encrypt each instruction pointer
        encrypted_ips[i] = encrypt_instruction_pointer(i, master_key)

    # Encrypt the labels
    encrypted_labels = {label: encrypted_ips[index] for label, index in labels.items()}

    print("\nUpdated key assignment for BEQ, CALL, and JMP:")
    # Second pass: Update keys for BEQ, CALL, and JMP
    for i, instr in enumerate(instructions):
        opcode = instr["instr"][0]
        if opcode == "BEQ":
            instr["Knext_true"] = instructions[labels[instr["instr"][1]]]["Kprev"]
            instr["Knext_false"] = instructions[i+1]["Kprev"]
            print(f"BEQ at instruction {i}:")
            print(f"  Knext_true: {instr['Knext_true'].hex()}")
            print(f"  Knext_false: {instr['Knext_false'].hex()}")
        elif opcode in ["CALL", "JMP"]:
            instr["Knext"] = instructions[labels[instr["instr"][1]]]["Kprev"]
            print(f"{opcode} at instruction {i}:")
            print(f"  Updated Knext: {instr['Knext'].hex()}")
        elif opcode == "RET":
            print(f"RET at instruction {i}:")
            print(f"  Knext (for instruction encryption): {instr['Knext'].hex()}")
            print(f"  Note: Actual Knext for register encryption will be determined at runtime from the call stack")

    print("\nEncrypted instructions:")
    # Third pass: Encrypt instructions
    for i, instr in enumerate(instructions):
        if instr["instr"][0] == "BEQ":
            message = f"{instr['instr']}||{instr['Kprev'].hex()}||{instr['Knext_true'].hex()}||{instr['Knext_false'].hex()}".encode()
        elif instr["instr"][0] == "RET":
            message = f"{instr['instr']}||{instr['Kprev'].hex()}||DYNAMIC_NEXT_KEY".encode()
        else:
            message = f"{instr['instr']}||{instr['Kprev'].hex()}||{instr['Knext'].hex()}".encode()

        print(f"Instruction {i}: Encrypting data: {message}")
        instr["encrypted_instr"] = encrypt_with_mac(message, master_key)
        print(f"  Encrypted: {instr['encrypted_instr'].hex()[:32]}...{instr['encrypted_instr'].hex()[-32:]}")
        #print(f"  Instruction {i}: {instr['encrypted_instr'].hex()[:32]}...{instr['encrypted_instr'].hex()[-32:]}")

    return instructions, encrypted_labels, encrypted_ips

def execute_instructions(instructions, encrypted_labels, encrypted_ips, master_key,initial_encrypted_memory):
    registers = {"AX": 0, "BX": 0, "CX": 0, "DX": 0}
    memory = initial_encrypted_memory
    instruction_ptr = 0
    call_stack = []  # Simple list instead of a class
    condition = None

    # Encrypt initial register state with master key
    registers = encrypt_registers(registers, master_key)
    current_key = master_key

    while instruction_ptr < len(instructions):
        # Fetch the encrypted instruction pointer
        encrypted_ip = encrypted_ips[instruction_ptr]

        # Fetch and decrypt the instruction
        instr_data = instructions[instruction_ptr]
        encrypted_instr = instr_data["encrypted_instr"]
        decrypted_data = decrypt_with_mac(encrypted_instr, master_key).decode()
        parts = decrypted_data.split("||")
        instr = eval(parts[0])
        kprev = bytes.fromhex(parts[1])
        
        if instr[0] == "BEQ":
            knext_true = bytes.fromhex(parts[2])
            knext_false = bytes.fromhex(parts[3])
        elif instr[0] == "RET":
            knext = "DYNAMIC"  # Will be determined from call stack
        else:
            knext = bytes.fromhex(parts[2])

        opcode = instr[0]

        # Decrypt registers using the current key
        decrypted_registers = decrypt_registers(registers, current_key)
        # Decrypt memory using the master key
        decrypted_memory = decrypt_memory(memory, master_key)
        registers_before = decrypted_registers.copy()
        
        # Execute instruction
        if opcode == "MOV":
            mov(decrypted_registers, *instr[1:])
        elif opcode == "ADD":
            add(decrypted_registers, *instr[1:])
        elif opcode == "SUB":
            subtract(decrypted_registers, *instr[1:])
        elif opcode == "MUL":
            multiply(decrypted_registers, *instr[1:])
        elif opcode == "DIV":
            divide(decrypted_registers, *instr[1:])
        elif opcode == "XOR":
            xor(decrypted_registers, *instr[1:])
        elif opcode == "AND":
            logical_and(decrypted_registers, *instr[1:])
        elif opcode == "OR":
            logical_or(decrypted_registers, *instr[1:])
        elif opcode == "NOT":
            logical_not(decrypted_registers, *instr[1:])
        elif opcode == "SHL":
            shl(decrypted_registers, *instr[1:])
        elif opcode == "SHR":
            shr(decrypted_registers, *instr[1:])
        elif opcode == "INC":
            inc(decrypted_registers, *instr[1:])
        elif opcode == "DEC":
            dec(decrypted_registers, *instr[1:])
        elif opcode == "NOP":
            nop(decrypted_registers)
        elif opcode == "LOAD":
            load(decrypted_registers, *instr[1:], decrypted_memory)
            print(f"\nAfter LOAD from address {instr[2]}:")
            print(f"  {instr[1]} = {decrypted_registers[instr[1]]}")
        elif opcode == "STORE":
            store(decrypted_registers, *instr[1:], decrypted_memory)
            print(f"\nAfter STORE to address {instr[2]}:")
            print(f"  Address {instr[2]} = {decrypted_memory[instr[2]]}")
        elif opcode == "CMP":
            condition = cmp(decrypted_registers, *instr[1:])
        elif opcode == "BEQ":
            if condition:
                encrypted_ip = encrypted_labels[instr[1]]
                next_key = knext_true
            else:
                encrypted_ip = encrypted_ips[instruction_ptr + 1]
                next_key = knext_false
            registers = encrypt_registers(decrypted_registers, next_key)
            memory = encrypt_memory(decrypted_memory, master_key)
            log_instruction(instr, registers_before, registers, current_key, next_key)
            current_key = next_key
            instruction_ptr = decrypt_instruction_pointer(encrypted_ip, master_key)
            continue
        elif opcode == "CALL":
            next_encrypted_ip = encrypted_ips[instruction_ptr + 1]
            # Push directly to the stack
            encrypted_data = encrypt_with_mac(next_encrypted_ip + knext, master_key)
            call_stack.append(encrypted_data)
            
            encrypted_ip = encrypted_labels[instr[1]]
            instruction_ptr = decrypt_instruction_pointer(encrypted_ip, master_key)
            next_key = instructions[instruction_ptr]["Kprev"]
            registers = encrypt_registers(decrypted_registers, next_key)
            memory = encrypt_memory(decrypted_memory, master_key)
            log_instruction(instr, registers_before, registers, current_key, next_key)
            current_key = next_key
            continue

        elif opcode == "RET":
            if call_stack:
                # Pop and decrypt directly
                encrypted_data = call_stack.pop()
                decrypted_data = decrypt_with_mac(encrypted_data, master_key)
                encrypted_return_ip = decrypted_data[:-KEY_SIZE]
                return_key = decrypted_data[-KEY_SIZE:]
                
                instruction_ptr = decrypt_instruction_pointer(encrypted_return_ip, master_key)
                registers = encrypt_registers(decrypted_registers, return_key)
                memory = encrypt_memory(decrypted_memory, master_key)
                log_instruction(instr, registers_before, registers, current_key, return_key)
                current_key = return_key
                continue
        elif opcode == "JMP":
            encrypted_ip = encrypted_labels[instr[1]]
            instruction_ptr = decrypt_instruction_pointer(encrypted_ip, master_key)
            next_key = instructions[instruction_ptr]["Kprev"]
            registers = encrypt_registers(decrypted_registers, next_key)
            memory = encrypt_memory(decrypted_memory, master_key)
            log_instruction(instr, registers_before, registers, current_key, next_key)
            current_key = next_key
            continue

        # Encrypt registers with Knext
        registers = encrypt_registers(decrypted_registers, knext)
        # Encrypt memory with master_key
        memory = encrypt_memory(decrypted_memory, master_key)

        log_instruction(instr, registers_before, registers, current_key, knext)
        current_key = knext

        # For normal sequential execution, move to the next encrypted instruction pointer
        if instruction_ptr + 1 < len(instructions):
            encrypted_ip = encrypted_ips[instruction_ptr + 1]
            instruction_ptr = decrypt_instruction_pointer(encrypted_ip, master_key)
        else:
            break

    # Final decryption of registers using the current key
    final_registers = decrypt_registers(registers, current_key)
    # Final decryption of memory using the master key
    final_memory = decrypt_memory(memory, master_key)

    return final_registers, final_memory

# Main with Timing
def main():
    master_key = generate_key()
    print(f"Master Key: {master_key.hex()}\n")

    # Initialize memory with some values
    initial_memory = {
        100: 5,    # Address 100 contains value 5
        200: 15,   # Address 200 contains value 15
        300: 50,   # Address 300 contains value 50
        400: 100   # Address 400 contains value 100
    }
    print("\nInitial Memory:")
    for addr, value in initial_memory.items():
        print(f"  Address {addr}: {value}")

    # Encrypt the initial memory
    initial_encrypted_memory = encrypt_memory(initial_memory, master_key)
    print("\nEncrypted Memory (showing first 16 bytes of each encrypted value):")
    for addr, encrypted_value in initial_encrypted_memory.items():
        print(f"  Address {addr}: {encrypted_value.hex()[:32]}...")

    initial_instructions = [
       {"instr": ("MOV", "AX", 10)},
        {"instr": ("MOV", "BX", 3)},
        {"instr": ("CALL", "subtract")},
        {"instr": ("MOV", "CX", "AX")},
        {"instr": ("CALL", "subtract")},
        {"instr": ("MOV", "DX", "AX")},
        {"instr": ("NOP",)},
        {"instr": ("CMP", "DX", 1)},
        {"instr": ("BEQ", "equal_one")},
        {"instr": ("MOV", "DX", 0)},
        {"instr": ("equal_one:",)},
        {"instr": ("MOV", "DX", 100)},
        {"instr": ("JMP", "end")},
        {"instr": ("NOP",)},
        {"instr": ("subtract:",)},
        {"instr": ("SUB", "AX", "BX")},
        {"instr": ("RET",)},
        {"instr": ("end:",)} 
    ]

    instructions, encrypted_labels, encrypted_ips = prepare_instructions(initial_instructions, master_key)
    
    print("\nStarting execution:")
    final_registers, final_memory = execute_instructions(instructions, encrypted_labels, encrypted_ips, master_key, initial_encrypted_memory)

    print("\nFinal Registers:")
    for reg, value in final_registers.items():
        print(f"  {reg}: {value}")

    print("\nFinal Memory (non-zero values):")
    for addr, value in final_memory.items():
        if value != 0:
            print(f"  Address {addr}: {value}")

    # 1️⃣ Primitives
    benchmark_primitives()

    # 2️⃣ Memory encryption throughput
    initial_memory = {100:5, 200:15, 300:50, 400:100}
    total_bytes = len(initial_memory) * 8  # 8 bytes per value

    t0 = time.perf_counter()
    enc_mem = encrypt_memory(initial_memory, master_key)
    t1 = time.perf_counter()
    mem_time = t1 - t0
    thr = total_bytes / mem_time
    print(f"\n[Memory Encrypt] {total_bytes} B in {mem_time:.4f}s ⇒ {thr:.1f} B/s")

    # 3️⃣ Instruction Preparation
    from copy import deepcopy
    # reuse initial_instructions exactly as before
    instrs_copy = deepcopy(initial_instructions)
    t0 = time.perf_counter()
    instructions, labels, ips = prepare_instructions(instrs_copy, master_key)
    t1 = time.perf_counter()
    print(f"[Instr Prep] {len(instructions)} instr in {(t1-t0):.4f}s")

    # 4️⃣ Instruction Execution
    t0 = time.perf_counter()
    final_regs, final_mem = execute_instructions(
        instructions, labels, ips, master_key, enc_mem
    )
    t1 = time.perf_counter()
    print(f"[Execution] {len(instructions)} instr in {(t1-t0):.4f}s\n")

    # 5️⃣ Final state
    print("Final Registers:", final_regs)
    print("Final Memory:", {a:v for a,v in final_mem.items() if v})

if __name__ == "__main__":
    main()

Master Key: c0cfb74a08ce2a0e1fefd2293bc6436ecba7fddfb1e9e450e44d4030feb28709


Initial Memory:
  Address 100: 5
  Address 200: 15
  Address 300: 50
  Address 400: 100

Encrypted Memory (showing first 16 bytes of each encrypted value):
  Address 100: 3c6a96ce1261efb13768f252946860a2...
  Address 200: 33c9f8996ab4ad94ed3fdee34196f497...
  Address 300: c949184c936c54d65a05c413c26d8c3e...
  Address 400: 733d8aa57c6d1184100665b9a1941898...

Initial key assignment:
Instruction 0: ('MOV', 'AX', 10)
  Kprev: c0cfb74a08ce2a0e1fefd2293bc6436ecba7fddfb1e9e450e44d4030feb28709
  Knext: b394f2e038abd0bee25fa108aeceb916082c898a698eeda26840cfea2c2ad57c
Instruction 1: ('MOV', 'BX', 3)
  Kprev: b394f2e038abd0bee25fa108aeceb916082c898a698eeda26840cfea2c2ad57c
  Knext: f8b2406e7519d344a70cc2fa0a5f1725db56e80467b0b6d624f7c400ee9ec67c
Instruction 2: ('CALL', 'subtract')
  Kprev: f8b2406e7519d344a70cc2fa0a5f1725db56e80467b0b6d624f7c400ee9ec67c
  Knext: 1dae88274b072a1aef787c1b028819b867f3cfc31b24b72e64b1822c